In [1]:
using HomotopyContinuation

# Witness Sets

**Witness sets** form the fundamental data type in numerical algebraic geomtry - they are how we represent positive dimensional varieties

A *witness set* for an irreducible variety $X \subset \mathbb{C}^n$ is a triple 
$$W=(F,L,S)$$
where 
- $\color{blue}{F}$ is a polynomial system for which $X$ is a component, 
- $\color{green}{L}$ is a generic affine linear space of complementary dimension
- $\color{red}{S} = X \cap L$

<img src="witnesscubic.png" width="300">

If $X$ is not irreducible, but equidimensional, we call 
$$W=(F,L,S)$$
a *witness superset* when $X$ is a union of components of $\mathcal V(F)$ and $L$ and $S$ are the same as above.


Note! $\{(x,L) \mid F(x)=L(x)=0\} \to \text{Grassmannian}$ is a branched cover, so everything we discussed before applies

In [165]:
@var x y z
TwistedCubic=System([y^2-x*z,x*y-z,x^2-y])

System of length 3
 3 variables: x, y, z

 -x*z + y^2
 -z + x*y
 -y + x^2

In [166]:
# witness_set will compute a witness superset for the dimension (numvars-numeqs) part of V(F), 
#    unless you actually give it a codimension
#W_twisted=witness_set(TwistedCubic)
W_twisted=witness_set(TwistedCubic; codim=2)

Witness set for dimension 1 of degree 3

In particular, notice that witness sets inherently encode the dimension and degree

In [167]:
# If we only take 2 of the equations of the twisted cubic, we get that extra line
TwistedCubic2 = System([y^2-x*z,x*y-z])
W_twisted2=witness_set(TwistedCubic2) #warning: a witness superset

Witness set for dimension 1 of degree 4

In [61]:
W_twisted2.L # the linear space we cut with

2-dim. affine linear subspace {x | Ax=b} with eltype Complex{Float64}:
A:
Complex{Float64}[-0.5935615382682724 - 0.5874685992944196im -0.2879650931631233 + 0.09876036615459721im 0.35927538157882255 - 0.2842693098447488im]
b:
Complex{Float64}[-0.20718652247693767 + 0.2592723245019178im]

In [168]:
W_twisted2.R # the solutions

4-element Array{PathResult,1}:
 PathResult:
 • return_code → :success
 • solution → Complex{Float64}[0.6521745371553913 + 1.034070335828349im, -0.6439698325263056 + 1.348788685309947im, -1.8147230962498124 + 0.213735535578641im]
 • accuracy → 2.8354e-16
 • residual → 3.6133e-16
 • condition_jacobian → 7.0672
 • steps → 58 / 0
 • extended_precision → false
 • path_number → 1

 PathResult:
 • return_code → :success
 • solution → Complex{Float64}[0.3023372746996572 - 0.22541310502177747im, 0.04059675975725709 - 0.13630156770774363im, -0.018450245889692695 - 0.05036008618875647im]
 • accuracy → 6.6147e-17
 • residual → 1.0945e-17
 • condition_jacobian → 9.5915
 • steps → 66 / 0
 • extended_precision → false
 • path_number → 2

 PathResult:
 • return_code → :success
 • solution → Complex{Float64}[-0.4278366122954059 - 0.3675849864426419im, 0.04792544456237225 + 0.3145326306605453im, 0.09511321293280843 - 0.15218524904788702im]
 • accuracy → 1.1841e-16
 • residual → 2.7097e-17
 • condition_j

In [169]:
#calling monodromy_solve with an underdetermined system + a dimension assumes you want witness points
# one issue is that your variety may be reducible

M=monodromy_solve(TwistedCubic2; dim=1) 

MonodromyResult
• return_code → :heuristic_stop
• 1 solutions
• 5 tracked loops
• random_seed → 0xe7d9b29b
• trace → 0.022318563525307176

**Theorem:** Monodromy of $(x,L) \in \mathcal V(F) \cap L \to L$ is the full symmetric group $S_{deg(\mathcal V(F))}$ if and only if $V(F)$ is irreducible. Otherwise, it is the product of symmetric groups for each irreducible component. 

This is why if we start monodromy at a point on the line, we can't get to a point on the twisted cubic.

In [170]:
MS_list=[monodromy_solve(TwistedCubic2; dim=1) for i in 1:30]

30-element Array{MonodromyResult{LinearSubspace{Complex{Float64}},LinearSubspace{Complex{Float64}}},1}:
 MonodromyResult
• return_code → :success
• 3 solutions
• 9 tracked loops
• random_seed → 0x941575c7
• trace → 6.253285507712339e-17
 MonodromyResult
• return_code → :success
• 3 solutions
• 6 tracked loops
• random_seed → 0x1ac83f16
• trace → 7.015851909750701e-17
 MonodromyResult
• return_code → :success
• 3 solutions
• 9 tracked loops
• random_seed → 0xfbaf9fe4
• trace → 6.505651488767584e-17
 MonodromyResult
• return_code → :success
• 3 solutions
• 6 tracked loops
• random_seed → 0x4f8a058b
• trace → 1.0794549116922288e-17
 MonodromyResult
• return_code → :success
• 3 solutions
• 24 tracked loops
• random_seed → 0xa28ff0d5
• trace → 3.683836174454106e-17
 MonodromyResult
• return_code → :success
• 3 solutions
• 3 tracked loops
• random_seed → 0x29dde00a
• trace → 9.117106081132545e-18
 MonodromyResult
• return_code → :success
• 3 solutions
• 15 tracked loops
• random_seed → 0x944

Notice that `return_code` can either be `success` or `heuristic_stop` (in our previous monodromy lecture, we only saw `heuristic_stop`)

### Trace test
`success` means it satisfied some given stopping criterion. In the case of witness sets, this is the **trace test**

The **trace** of a subset of points in $\mathbb{C}^n$ is their coordinate-wise average.

**Lemma**: Let $X$ be an irreducible variety and $L_t$ be a parallel family of affine linear spaces of complementary dimension. Let $S_0 \subset X \cap L_0$ and $S_t$ its continuation as $L_t$ moves. Then the trace of $S_t$ move linearly if and only if $S_t = X \cap L_0$.


<img src="TraceExample1.png" width="600">

Now let's try moving a witness set and verifying this ourselves

In [37]:
W_twisted2
#Here is the normal vector to the hyperplane L in our witness set W_twisted2=(F,L,S)
W_twisted2.L.extrinsic.A

1×3 Array{Complex{Float64},2}:
 -0.101215-0.534835im  -0.746532+0.375746im  0.0623277-0.0364253im

In [67]:
L₁ = W_twisted2.L
L₂ = LinearSubspace(L₁.extrinsic.A, L₁.extrinsic.b+[1])
L₃ = LinearSubspace(L₁.extrinsic.A, L₁.extrinsic.b+[2]);

In [73]:
trace₁=sum(solutions(witness_set(W_twisted2,L₁)))
println("First trace: \n",trace₁)
trace₂=sum(solutions(witness_set(W_twisted2,L₂)))
println("Second trace: \n",trace₂)
trace₃=sum(solutions(witness_set(W_twisted2,L₃)))
println("Third trace: \n",trace₃)

First trace: 
Complex{Float64}[0.5846205092777977 - 0.1742132031244481im, 0.7846457146465493 + 3.8959697965649176im, -2.748293583288453 + 4.2871991765375475im]
Second trace: 
Complex{Float64}[-0.2664435067199723 + 0.6681145980447039im, 0.7846457146465498 + 3.895969796564917im, 2.3869545713973315 + 8.350359425366838im]
Third trace: 
Complex{Float64}[-1.1175075227177427 + 1.510442399213856im, 0.7846457146465502 + 3.895969796564917im, 7.5222027260831155 + 12.41351967419613im]


In [74]:
# here we check trace2 is the midpoint of trace1 and trace3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
 -3.3306690738754696e-16 + 2.220446049250313e-16im
                     0.0 + 0.0im
                     0.0 + 0.0im

We can find out which solution is the one on the line by checking traces of subsets

In [80]:
I=[3]
trace₁=sum(solutions(witness_set(W_twisted2,L₁))[I])
trace₂=sum(solutions(witness_set(W_twisted2,L₂))[I])
trace₃=sum(solutions(witness_set(W_twisted2,L₃))[I])

3-element Array{Complex{Float64},1}:
 0.6266838023996777 + 0.22096311051311313im
 0.7846457146465502 + 3.895969796564917im
 7.5222027260831155 + 12.41351967419613im

In [81]:
# here we check trace2 is the midpoint of trace1 and trace 3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
 -2.220446049250313e-16 + 3.3306690738754696e-16im
                    0.0 + 0.0im
                    0.0 + 0.0im

### So what can we do with witness sets?

1) Move $(F,L,S) \to (F,L',S')$

2) Sample from $X$

3) Test membership for $x \in X$


#### Moving a witness set (this truly is the fundamental operation which makes everything work)

In [96]:
L = LinearSubspace([1,2,3]',[5])

2-dim. affine linear subspace {x | Ax=b} with eltype Float64:
A:
[-0.2672612419124243 -0.5345224838248488 -0.8017837257372732]
b:
[-1.3363062095621219]

In [99]:
newW=witness_set(W_twisted,L)

Witness set for dimension 1 of degree 3

#### We can also move it to a special slice (but this technically is not a witness set if we don't have the correct # of points)

In [104]:
special_Lx = LinearSubspace([0,1,3]',[2])
specialSlicex=witness_set(W_twisted,special_Lx)

Witness set for dimension 1 of degree 3

In [105]:
special_Ly = LinearSubspace([2,0,4]',[3])
specialSlicey=witness_set(W_twisted,special_Ly)

Witness set for dimension 1 of degree 3

In [107]:
#Warning: not a witness set
special_Lz = LinearSubspace([3,-2,0]',[3])
specialSlicez=witness_set(W_twisted,special_Lz)

Witness set for dimension 1 of degree 2

In [112]:
#And if we try to move from this slice, we won't get the right answer
moveback=witness_set(specialSlicez,special_Ly)

2-element Array{Array{Complex{Float64},1},1}:
 [-0.36404106153397714 - 0.9474057649435931im, -0.7650517889655698 + 0.6897892007469505im, 0.9320205307669887 + 0.47370288247179654im]
 [-0.36404106153397714 + 0.9474057649435931im, -0.7650517889655698 - 0.6897892007469505im, 0.9320205307669887 - 0.47370288247179654im]

#### Sampling points

In [128]:
length(solutions(W_twisted)[1])

3

In [133]:
function sample(W::WitnessSet)
    n=length(solutions(W)[1]) #number of variables
    randomLinear = LinearSubspace(randn(ComplexF64,dim(W),n),randn(ComplexF64,dim(W)))
    newWitness=witness_set(W,randomLinear)
    s=solutions(newWitness)[1]
    return s
end

sample (generic function with 1 method)

In [139]:
sample(W_twisted)

3-element Array{Complex{Float64},1}:
  1.457236954966447 - 0.11384846101564677im
 2.1105780708442516 - 0.3318083693161147im
 3.0378364889772844 - 0.7238094829535879im

#### Membership

In [141]:
A=randn(ComplexF64,1,3)
A*sample(W_twisted)

1-element Array{Complex{Float64},1}:
 0.346306699053596 - 0.22843871867509266im

In [150]:
function membership(W::WitnessSet,p)
    n=length(solutions(W)[1]) #number of variables
    A=randn(ComplexF64,dim(W),n)
    randomLinearThroughP = LinearSubspace(A,A*p)
    newWitness=witness_set(W,randomLinearThroughP)
    check=findfirst(x->x≈p,solutions(newWitness))
    if check==nothing
        return false
    else
        return true
    end
end
    

membership (generic function with 1 method)

In [151]:
membership(W_twisted,[2,4,8])

true

In [153]:
#here we see again which solution in the twistedcubic+line is not on the twisted cubic
for s in solutions(W_twisted2)
    println(membership(W_twisted,s))
end

true
true
false
true


### Why a linear intersection?

If you have a strong background in algebraic geometry, you may like to think of a witness set as representing some intersection class in projective space. This viewpoint is explored in

*General witness sets for numerical algebraic geometry* Frank Sottile, 2020 https://arxiv.org/pdf/2002.00180.pdf


### Why is this so powerful?

One of the main reasons: **pseudo-witness sets** are easy to compute

**Definition:** A pseudo witness set for an (irreducible) variety $X\subset \mathbb{C}^n$ is a quadruple 
$$(F,\pi,\pi^{-1}(L),S)$$
where 
- $\pi:\mathbb{C}^N \to \mathbb{C}^n$ (really this works with any map by factoring through the graph)
- $X$ is a component of $\pi(\mathcal{V}(F))$ of the same dimension as $\mathcal{V}(F)$
- $L$ is a linear space of complementary dimension (to $X$) in $\mathbb{C}^n$
- $S = \pi^{-1}(L) \cap \mathcal V(F)$ (note! this is in $\mathbb{C}^N$)

Below is a picture of the pseudo-witness set for the projection of the twisted cubic onto the $(x,y)$-plane

<img src="PseudoWitness.png" width="500">

The points in $S$ are those which are upstairs, but it is obvious that applying $\pi$ gives the points downstairs

Be careful about the following situation; you'll need to slice once upstairs.

<img src="Careful.png" width="500">


### For images of maps

If $\varphi:\mathbb{C}_y^d \to \mathbb{C}_x^n$ is some map, you can compute a pseudo witness set for $\overline{\varphi(\mathbb{C}^d)}$ by taking $F = (x_1-\varphi_1(y),\ldots,x_n-\varphi_n(y))$ and $\pi:\mathbb{C}^{d+n} \to \mathbb{C}^n$ sending $\pi(y,x)=x$.

Note: If you've studied symbolic computational algebraic geometry, this is elimination...which is hard

In [155]:
W_parabola = witness_set(System([y-x^2]))

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Witness set for dimension 1 of degree 2

The parabola is parametrized by the first two coordinates of the twisted cubic

In [161]:
println(W_parabola.L.extrinsic.A)
Slice_For_Cubic=LinearSubspace(hcat(W_parabola.L.extrinsic.A,[0]),W_parabola.L.extrinsic.b)
PseudoWitness=witness_set(W_twisted,Slice_For_Cubic)

Complex{Float64}[-0.649266982668212 + 0.5004085823698871im 0.5620539810837469 - 0.11017694066993688im]


Witness set for dimension 1 of degree 2

In [163]:
println([s[1:2] for s in solutions(PseudoWitness)])

Array{Complex{Float64},1}[[1.5834270985288863 - 1.131631007667107im, 1.2266526388419356 - 3.5837104061512948im], [-0.3029371480886612 + 0.49231854095540273im, -0.1506066300763651 - 0.2982831494964009im]]


In [164]:
println(solutions(W_parabola))

Array{Complex{Float64},1}[[1.5834270985288865 - 1.1316310076671072im, 1.2266526388419359 - 3.5837104061512957im], [-0.3029371480886612 + 0.49231854095540273im, -0.1506066300763651 - 0.2982831494964009im]]


You can move the witness set by moving the pseudo-witness set.

### Numerical irreducible decomposition (of equidimensional varieties)

One of the most important algorithms in numerical algebraic geometry is taking a polynomial systme $F$ and computing a witness set for every irreducible component of $\mathcal V(F)$. This is called a **numerical irreducible decomposition**. We will go through how it works for equidimensional varieties

#### Step 1: compute a witness superset 

$\mathcal V(F) \to (F,L,S)$

#### Step 2: 

Using monodromy, identify solutions as belonging to the same component until the trace test passes.

$p \in S \to S_p=\{q \in S \mid q \text{ was found via monodromy starting from } p\}$

#### Step 3: 

If $S \backslash \bigcup_{p \text{ done in step 2}} S_p \neq \emptyset$ pick some $q$ not identified and repeat step 2

#### Output: 

Set $W_p = (F,L,S_p)$ and output the union of all $W_p$

### Other things you can do with witness sets

Given a witness set $W$ for $X$ you can also...

- find a witness set for $X \cap \mathcal V(f)$
- recover equations for $X$
- compute the Newton polytope of $X$ (when $X$ is a hypersurface)
- describe the real points of $X$
- much more...